# Generating 2D data

In [ ]:
import scipy.io
import os
import matplotlib.pyplot as plt
import numpy as np
import h5py

import sys
sys.path.append("..") 
from cryoem.plots import plot_projection, plot_projections
from cryoem.noise import gaussian_noise

In [ ]:
# location of *.h5 files
data_dir = "../generated_data"
projections_filename = "ProjectionsAngles_ProjNber5000_AngCoverage0.5_AngShift1.57.h5"

data_path = os.path.join(data_dir, projections_filename)

In [ ]:
# load structures
# data = scipy.io.loadmat(data_path)
data = h5py.File(data_path, 'r')

In [ ]:
len(data["Projections"])

In [ ]:
len(data["Angles"])

In [ ]:
data["Projections"].shape

In [ ]:
data["Projections"][0].shape

## Projection and Noise

When tested, functions for plotting and noise generated are put inside the corresponding files called plots.py and noise.py. This way we can use them from different notebooks, and not repeating ourselves.

### One projection without noise

In [ ]:
datapoint = 0
image = data["Projections"][datapoint]
angles = [ float(f"{x:.2f}") for x in data["Angles"][datapoint] ]
title = f'Projection {datapoint}\nAngles {angles}'
    
plot_projection(image, title)

### One projection with Gaussian noise

In [ ]:
datapoint = 0
image = data["Projections"][datapoint]
image = image + gaussian_noise(image.shape, mean=0, var=15)
angles = [ float(f"{x:.2f}") for x in data["Angles"][datapoint] ]
title = f'Projection {datapoint} with noise\nAngles {angles}'
    
plot_projection(image, title, mean=0, var=100)

### Several projections without noise

In [ ]:
nrows, ncols = 2, 5
start_row, start_col = 0, 0

images = []
titles = []

cr = [(i, j) for i in range(nrows) for j in range(ncols)]

for i, j in cr:
    datapoint = (start_row + i)*(start_col + ncols) + j    

    image = data["Projections"][datapoint]
    image = image + gaussian_noise(shape=image.shape, mean=0, var=0)

    angles = [ float(f"{x:.2f}") for x in data["Angles"][datapoint] ]
    title = f'Projection {datapoint}\nAngles {angles}'
    
    images.append(image)
    titles.append(title)
        
plot_projections(images, titles, nrows=nrows, ncols=ncols)

### Several projections with Gaussian noise

In [ ]:
nrows, ncols = 2, 5
start_row, start_col = 0, 0

images = []
titles = []

cr = [(i, j) for i in range(nrows) for j in range(ncols)]

for i, j in cr:
    datapoint = (start_row + i)*(start_col + ncols) + j    

    image = data["Projections"][datapoint]
    image = image + gaussian_noise(shape=image.shape, mean=0, var=10)

    angles = [ float(f"{x:.2f}") for x in data["Angles"][datapoint] ]
    title = f'Projection {datapoint} with noise\nAngles {angles}'
    
    images.append(image)
    titles.append(title)
        
plot_projections(images, titles, nrows=nrows, ncols=ncols)

---

# Library for 3D ploting from 2D

Package [ipyvolume](https://github.com/maartenbreddels/ipyvolume)

In [ ]:
import ipyvolume as ipv
import numpy as np

## Angles in Euclidean coordinate system (no rotation)

In [ ]:
x = np.array([ ang[0]for ang in data["Angles"]  ]) #x.flatten()
y = np.array([ ang[1]for ang in data["Angles"]  ]) #y.flatten()
z = np.array([ ang[2]for ang in data["Angles"]  ]) #r.flatten()
print("and flattened of shape", x.shape)

In [ ]:
max(x), max(y), max(z), 2*np.pi-np.pi/2

In [ ]:
min(x), min(y), min(z), np.pi/2

In [ ]:
# draw the scatter plot, and add controls with animate_glyphs
ipv.figure(width=1000, height=700)
s = ipv.scatter(x, z, y, marker="sphere")
#ipv.animation_control(s, interval=200)
#ipv.ylim(-3,3)
ipv.show()

## Camera positions

In [ ]:
from cryoem.projections import RotationMatrix

vectors = RotationMatrix(data["Angles"])

In [ ]:
# draw the scatter plot, and add controls with animate_glyphs
ipv.figure(width=1000, height=700)
s = ipv.scatter(vectors[:,0], vectors[:,2], vectors[:,1], marker="sphere")
#ipv.animation_control(s, interval=200)
#ipv.ylim(-3,3)
ipv.show()